In [37]:
# import the necessary packages
import numpy as np
import tensorflow as tf

In [26]:
# Dense class
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation

        # set the weight
        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)

        # set the bias
        b_shape = (output_size,)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)

    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)

    @property
    def weights(self):
        return [self.W, self.b]


In [27]:
# sequential class

class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x

    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

In [28]:
# batch generator class
import math


class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)

    def next(self):
        images = self.images[self.index: self.index + self.batch_size]
        labels = self.labels[self.index: self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

In [29]:
# instantiate the model
model = NaiveSequential([
    NaiveDense(input_size=28 * 28,
               output_size=512,
               activation=tf.nn.relu),
    NaiveDense(input_size=512,
               output_size=10,
               activation=tf.nn.softmax)
])

assert len(model.weights) == 4

In [30]:
# update weights
lr = 1e-3


def update_weights(gradients: tf.Variable, weights: tf.Variable):
    for g, w in zip(gradients, weights):
        g: tf.Variable
        w: tf.Variable

        w.assign_sub(lr * g)


In [31]:
# one training step
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        preds = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, preds)
        avg_loss = tf.reduce_mean(per_sample_losses)
        gradients = tape.gradient(avg_loss, model.weights)
        update_weights(gradients, model.weights)
        return avg_loss


In [32]:
# fit function
def fit (model,images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print(f"[INFO] epoch {epoch_counter + 1}......")

        batch_gen = BatchGenerator(images, labels)
        for batch_counter in range(batch_gen.num_batches):
            images_batch, labels_batch = batch_gen.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"[INFO] loss at batch: {batch_counter + 1} : {loss: .2f}......")

In [34]:
# load the data
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

In [35]:
fit(model=model,
    images=train_images,
    labels=train_labels,
    epochs=10)

[INFO] epoch 1......
[INFO] loss at batch: 1 :  6.34......
[INFO] loss at batch: 101 :  2.25......
[INFO] loss at batch: 201 :  2.21......
[INFO] loss at batch: 301 :  2.09......
[INFO] loss at batch: 401 :  2.16......
[INFO] epoch 2......
[INFO] loss at batch: 1 :  1.91......
[INFO] loss at batch: 101 :  1.88......
[INFO] loss at batch: 201 :  1.82......
[INFO] loss at batch: 301 :  1.70......
[INFO] loss at batch: 401 :  1.77......
[INFO] epoch 3......
[INFO] loss at batch: 1 :  1.59......
[INFO] loss at batch: 101 :  1.58......
[INFO] loss at batch: 201 :  1.49......
[INFO] loss at batch: 301 :  1.41......
[INFO] loss at batch: 401 :  1.47......
[INFO] epoch 4......
[INFO] loss at batch: 1 :  1.33......
[INFO] loss at batch: 101 :  1.34......
[INFO] loss at batch: 201 :  1.22......
[INFO] loss at batch: 301 :  1.20......
[INFO] loss at batch: 401 :  1.25......
[INFO] epoch 5......
[INFO] loss at batch: 1 :  1.13......
[INFO] loss at batch: 101 :  1.16......
[INFO] loss at batch: 201

In [38]:
# evaluate
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"[INFO] accuracy: {matches.mean(): .2f}......")

[INFO] accuracy:  0.82......
